In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import string
import re
import emoji

In [2]:
def load_data():
    data = pd.read_csv('ganjar.csv')
    return data
df = load_data()
df.tail()

,Unnamed: 0,id,date,text,place,screen_name
3908,2377,1686005227399651328,2023-07-31 13:25:45,RT @lek_brewok: Urusan ama wong edan nanti aja...,Solo ~ Jawa tengah ~ Indonesia,sussssie
3909,2378,1686005150635425792,2023-07-31 13:25:27,RT @Jumianto_RK: Relawan Gapura Nusantara (RGN...,Indonesia,wil_ws_
3910,2382,1686005038316134400,2023-07-31 13:25:00,Dukungan jg perlu digalang.\nItu yg terus dila...,NaN,Jelli_cent
3911,2384,1686005014840709120,2023-07-31 13:24:54,"RT @03__nakula: Gubernur Jateng, Ganjar Pranow...","Semarang Barat, Indonesia",wongkalibanteng
3912,2389,1686004865850585089,2023-07-31 13:24:19,Ayah Ganjar Pranowo\nPresiden 2024🇮🇩\n\n*Seman...,"Makassar, Indonesia",gmc_sulsel


In [3]:
df = pd.DataFrame(df[['id', 'date', 'place', 'screen_name', 'text']])
df

,id,date,place,screen_name,text
0,1661196268318920705,2023-05-24 02:23:48,Indonesia,IDSahabatGanjar,Untuk memperkuat pondasi kemenangan untuk Ganj...
1,1661196173997395969,2023-05-24 02:23:26,"DKI Jakarta, Indonesia",DiniHdianti,"RT @LaingenLisa: ""Semoga Ganjar Pranowo bisa m..."
2,1661196167450099714,2023-05-24 02:23:24,Jakarta,ShintaSusan2,RT @ShintaSusan2: Belajar Reformasi Birokrasi ...
3,1661196146767962116,2023-05-24 02:23:19,NaN,sahabatganjar,Untuk memperkuat pondasi kemenangan untuk Ganj...
4,1661196142363938817,2023-05-24 02:23:18,Jakarta,ShintaSusan2,Belajar Reformasi Birokrasi bersama Ganjar Pra...
...,...,...,...,...,...
3908,1686005227399651328,2023-07-31 13:25:45,Solo ~ Jawa tengah ~ Indonesia,sussssie,RT @lek_brewok: Urusan ama wong edan nanti aja...
3909,1686005150635425792,2023-07-31 13:25:27,Indonesia,wil_ws_,RT @Jumianto_RK: Relawan Gapura Nusantara (RGN...
3910,1686005038316134400,2023-07-31 13:25:00,NaN,Jelli_cent,Dukungan jg perlu digalang.\nItu yg terus dila...
3911,1686005014840709120,2023-07-31 13:24:54,"Semarang Barat, Indonesia",wongkalibanteng,"RT @03__nakula: Gubernur Jateng, Ganjar Pranow..."


# CLEANSING

In [4]:
# Menghapus nantion username
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

# Menghapus @user
df['cleansing'] = np.vectorize(remove_pattern) (df['text'], "@[\w]*")
df

,id,date,place,screen_name,text,cleansing
0,1661196268318920705,2023-05-24 02:23:48,Indonesia,IDSahabatGanjar,Untuk memperkuat pondasi kemenangan untuk Ganj...,Untuk memperkuat pondasi kemenangan untuk Ganj...
1,1661196173997395969,2023-05-24 02:23:26,"DKI Jakarta, Indonesia",DiniHdianti,"RT @LaingenLisa: ""Semoga Ganjar Pranowo bisa m...","RT : ""Semoga Ganjar Pranowo bisa melanjutkan k..."
2,1661196167450099714,2023-05-24 02:23:24,Jakarta,ShintaSusan2,RT @ShintaSusan2: Belajar Reformasi Birokrasi ...,RT : Belajar Reformasi Birokrasi bersama Ganja...
3,1661196146767962116,2023-05-24 02:23:19,NaN,sahabatganjar,Untuk memperkuat pondasi kemenangan untuk Ganj...,Untuk memperkuat pondasi kemenangan untuk Ganj...
4,1661196142363938817,2023-05-24 02:23:18,Jakarta,ShintaSusan2,Belajar Reformasi Birokrasi bersama Ganjar Pra...,Belajar Reformasi Birokrasi bersama Ganjar Pra...
...,...,...,...,...,...,...
3908,1686005227399651328,2023-07-31 13:25:45,Solo ~ Jawa tengah ~ Indonesia,sussssie,RT @lek_brewok: Urusan ama wong edan nanti aja...,RT : Urusan ama wong edan nanti aja.. ane masi...
3909,1686005150635425792,2023-07-31 13:25:27,Indonesia,wil_ws_,RT @Jumianto_RK: Relawan Gapura Nusantara (RGN...,RT : Relawan Gapura Nusantara (RGN) Purnawiraw...
3910,1686005038316134400,2023-07-31 13:25:00,NaN,Jelli_cent,Dukungan jg perlu digalang.\nItu yg terus dila...,Dukungan jg perlu digalang.\nItu yg terus dila...
3911,1686005014840709120,2023-07-31 13:24:54,"Semarang Barat, Indonesia",wongkalibanteng,"RT @03__nakula: Gubernur Jateng, Ganjar Pranow...","RT : Gubernur Jateng, Ganjar Pranowo minta aga..."


In [5]:
def get_emoji_regexp():
    # Sort emoji by length to make sure multi-character emojis
    emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
    pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
    return re.compile(pattern)

def remove(tweet):
    # RT
    tweet = re.sub(r'RT[\s]+', '', tweet)
    # URLs
    tweet = re.sub(r'https\S+', '', tweet)
    tweet = re.sub(r'http[s]?\:\/\/.[a-zA-Z0-9\.\/\_?=%&#\-\+!]+',' ', tweet)
    tweet = re.sub(r'pic.twitter.com?.[a-zA-Z0-9\.\/\_?=%&#\-\+!]+',' ', tweet)
    # Numbers
    tweet = re.sub(r'[0-9]+', '', tweet)
    tweet = re.sub("\W+", " ", tweet)
    tweet = re.sub(r'\\n', " ", tweet)
    # Simbol
    tweet = re.sub(r'[!$%^&*@#()_+|~=`{}\[\]%\-:";\'<>?,.\/]', ' ', tweet)
    # Koreksi duplikasi tiga karakter beruntun atau lebih (contoh. yukkk)
    tweet = re.sub(r'([a-zA-Z])\1\1','\\1', tweet)
    # Emoji
    tweet = get_emoji_regexp().sub("", tweet).strip()
    # dibawah 3 huruf
    tweet = re.sub(r'\b\w{1,3}\b', '', tweet)
    # Spasi ganda (atau lebih) menjadi satu spasi
    tweet = re.sub(' +', ' ', tweet)
    return tweet

def remove_whitespace(tweet):
    return ' '.join(tweet.split())

def tokenizing (tweet):
    tokens = word_tokenize(str.remove(tweet))
    return ' ', join(tokens)

df['cleansing'] = df['cleansing'].apply(lambda x: remove(x))

In [6]:
df.head()

,id,date,place,screen_name,text,cleansing
0,1661196268318920705,2023-05-24 02:23:48,Indonesia,IDSahabatGanjar,Untuk memperkuat pondasi kemenangan untuk Ganj...,Untuk memperkuat pondasi kemenangan untuk Ganj...
1,1661196173997395969,2023-05-24 02:23:26,"DKI Jakarta, Indonesia",DiniHdianti,"RT @LaingenLisa: ""Semoga Ganjar Pranowo bisa m...",Semoga Ganjar Pranowo bisa melanjutkan kepemim...
2,1661196167450099714,2023-05-24 02:23:24,Jakarta,ShintaSusan2,RT @ShintaSusan2: Belajar Reformasi Birokrasi ...,Belajar Reformasi Birokrasi bersama Ganjar Pra...
3,1661196146767962116,2023-05-24 02:23:19,NaN,sahabatganjar,Untuk memperkuat pondasi kemenangan untuk Ganj...,Untuk memperkuat pondasi kemenangan untuk Ganj...
4,1661196142363938817,2023-05-24 02:23:18,Jakarta,ShintaSusan2,Belajar Reformasi Birokrasi bersama Ganjar Pra...,Belajar Reformasi Birokrasi bersama Ganjar Pra...


# CASE FOLD

In [7]:
def uppercase():
    data = df['cleansing'].str.lower()
    return data

df['case fold'] = uppercase()
results = df['case fold']
print(results)

0       untuk memperkuat pondasi kemenangan untuk ganj...
1       semoga ganjar pranowo bisa melanjutkan kepemim...
2       belajar reformasi birokrasi bersama ganjar pra...
3       untuk memperkuat pondasi kemenangan untuk ganj...
4       belajar reformasi birokrasi bersama ganjar pra...
                              ...                        
3908    urusan wong edan nanti masih urusan sama pasuk...
3909    relawan gapura nusantara purnawirawan polri me...
3910    dukungan perlu digalang terus dilakukan sahaba...
3911    gubernur jateng ganjar pranowo minta agar para...
3912    ayah ganjar pranowo presiden semangat mengabdi...
Name: case fold, Length: 3913, dtype: object


# STOPWORDS

In [8]:
# memanfaatkan modul stopwords NLTK untuk menggunakan stopwords kustom
nltk.download('stopwords')
stopword_nltk = stopwords.words('indonesian')
more_stopword = ['anies', 'baswedan', 'ganjar', 'pranowo', 'prabowo', 'subianto']

# Sastrawi
factory = StopWordRemoverFactory()
# stopword = factory.create_stop_word_remover()
stopwords = factory.get_stop_words()+stopword_nltk+more_stopword
print(stopwords)

['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wols88\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def menghapusKata(tweet):
    clean_list = [word for word in tweet.split() if word not in stopwords]
    return clean_list

tweet_hapus = results.apply(menghapusKata)
df['stopwords'] = tweet_hapus
print(tweet_hapus)

0       [memperkuat, pondasi, kemenangan, sahabat, dep...
1       [semoga, melanjutkan, kepemimpinan, nasional, ...
2       [belajar, reformasi, birokrasi, menangtotal, b...
3       [memperkuat, pondasi, kemenangan, sahabat, dep...
4       [belajar, reformasi, birokrasi, menangtotal, b...
                              ...                        
3908    [urusan, edan, urusan, pasukan, sopir, becak, ...
3909    [relawan, gapura, nusantara, purnawirawan, pol...
3910    [dukungan, digalang, sahabatganjar, memenangka...
3911    [gubernur, jateng, remaja, nikah, muda, salah,...
3912    [ayah, presiden, semangat, mengabdi, membangun...
Name: case fold, Length: 3913, dtype: object


# STEMMING

In [10]:
factory = StemmerFactory()
stem_say = factory.create_stemmer()

def stemmed_wrapper(term):
    return stem_say.stem(term)

term_base = {}

for document in tweet_hapus:
    for term in document:
        if term not in term_base:
            term_base[term] = " "
            
print(len(term_base))
print("--------------")

for term in term_base:
    term_base[term] = stemmed_wrapper(term)
    print(term,":", term_base[term])
    
print(term_base)
print("-------------")

def get_stemmed_term(document):
    return [term_base[term] for term in document]

stem_done = tweet_hapus.apply(get_stemmed_term)

print(stem_done)

6186
--------------
memperkuat : kuat
pondasi : pondasi
kemenangan : menang
sahabat : sahabat
depok : depok
berkunjung : kunjung
kota : kota
semoga : moga
melanjutkan : lanjut
kepemimpinan : pimpin
nasional : nasional
presiden : presiden
joko : joko
widodo : widodo
belajar : ajar
reformasi : reformasi
birokrasi : birokrasi
menangtotal : menangtotal
becca : becca
hasto : hasto
smtown : smtown
abel : abel
cristiano : cristiano
crist : crist
bukti : bukti
relawan : rawan
gagal : gagal
polisikan : polisi
pembangunan : bangun
jalan : jalan
jokowi : jokowi
temponasional : temponasional
gubernur : gubernur
jawa : jawa
jateng : jateng
mengunggah : unggah
momen : momen
makan : makan
malam : malam
wali : wali
surakarta : surakarta
gibran : gibran
raka : raka
sesungguhnya : sungguh
elektabilitas : elektabilitas
capres : capres
pdip : pdip
puan : puan
maharani : maharani
retweet : retweet
ganja : ganja
survei : survei
periodik : periodik
litbang : litbang
kompas : kompas
wawancara : wawancara
tata

sartu : sartu
periode : periode
kabinet : kabinet
sejatinya : sejati
teng : teng
hadir : hadir
upacara : upacara
pelepasan : lepas
peserta : serta
didik : didik
kelas : kelas
kesempatan : sempat
prano : prano
menangkanganjar : menangkanganjar
generasigoto : generasigoto
libur : libur
halbil : halbil
jumat : jumat
kerja : kerja
keliling : keliling
magelang : magelang
sabtu : sabtu
palembang : palembang
minggu : minggu
friendship : friendship
jaka : jaka
festival : festival
mengguncang : guncang
semarang : semarang
lapangan : lapang
pelaksanaan : laksana
pemilihan : pilih
bergerilya : gerilya
mengumpulkan : kumpul
masya : masya
ektp : ektp
yuhuu : yuhuu
tancap : tancap
jare : jare
jurkam : jurkam
heran : heran
etho : etho
memiliki : milik
kedekatan : dekat
ranah : ranah
profesional : profesional
anggaran : anggar
milliar : milliar
puskesma : puskesma
ngamuk : ngamuk
mega : mega
berdasarkan : dasar
dirilis : rilis
charta : charta
polytytyd : polytytyd
silitbang : silitbang
kompos : kompos

huhu : huhu
salam : salam
pancasil : pancasil
membara : bara
teriakan : ria
menggema : gema
pendukungnya : dukung
relaw : relaw
cilegon : cilegon
kerahkan : kerah
piliha : piliha
bnaten : bnaten
milenials : milenials
pergilah : pergi
jauhnya : jauh
terbanglah : terbang
daring : daring
menegur : tegur
teatrikal : teatrikal
pengendara : kendara
motor : motor
terj : terj
unik : unik
dialami : alami
sowan : sowan
yayasan : yayasan
pondok : pondok
pesantren : pesantren
miftahul : miftahul
nazar : nazar
gaboleh : gaboleh
main : main
terucap : ucap
jarak : jarak
hamid : hamid
bernazar : nazar
indo : indo
kyknya : kyknya
cinta : cinta
luntur : luntur
negri : negri
kelangsungan : langsung
kemajuan : maju
nkri : nkri
seandainya : anda
duet : duet
alun : alun
hihi : hihi
bantensehatbarengganjar : bantensehatbarengganjar
gelar : gelar
menghibur : hibur
dilanjutkan : lanjut
gpfestival : gpfestival
papan : papan
singgung : singgung
keok : keok
cengeng : cengeng
simpatisan : simpatisan
serang : seran

walubi : walubi
bakti : bakti
waisak : waisak
berdikari : berdikari
omong : omong
mimpi : mimpi
siang : siang
bolong : bolong
memanusiakan : manusia
mengajarkan : ajar
menghargai : harga
hargai : harga
mewakili : wakil
pernyataan : nyata
tangga : tangga
anggap : anggap
sepele : sepele
ternya : ter
kejujuran : jujur
warisan : waris
berharga : harga
keharuan : haru
wagyo : wagyo
muridnya : murid
bersekolah : sekolah
peduli : peduli
pendidikan : didik
meresmikan : resmi
ruang : ruang
hijau : hijau
leyangan : leyangan
rabu : rabu
penerusnya : terus
perhatkan : perhatkan
khusus : khusus
ganjaruntukindonesia : ganjaruntukindonesia
raya : raya
gila : gila
menahan : tahan
kebahagiannya : kebahagiannya
keinginannya : ingin
natural : natural
pencilakan : pencilakan
berterus : terus
cuci : cuci
ambil : ambil
tanggung : tanggung
semen : semen
remb : remb
teka : teka
teki : teki
nantikan : nanti
ganjarberikankemajuan : ganjarberikankemajuan
ganjarpilihanterbaik : ganjarpilihanterbaik
bersinar : sin

generas : generas
bilang : bilang
mencari : cari
pengganti : ganti
soso : soso
tongkat : tongkat
kepemimpi : mimpi
memotivasi : motivasi
akbar : akbar
hall : hall
ganjarjadikanrimakmurkuat : ganjarjadikanrimakmurkuat
puji : puji
inter : inter
milan : mil
diduetkan : duet
mematahkan : patah
dimainkan : main
rayu : rayu
selera : selera
lidah : lidah
lauk : lauk
ikan : ikan
asin : asin
dimasak : masak
menu : menu
kepribadian : pribadi
sayang : sayang
bocah : bocah
berbagi : bagi
pokomen : pokomen
capresku : capresku
persilakan : sila
cebong : cebong
whoever : whoever
whatever : whatever
kampanyekan : kampanye
lestarikan : lestari
perubahan : ubah
estafet : estafet
dihati : hati
dinanti : nanti
akui : aku
deklarasikan : deklarasi
entaskan : entas
kemiskinan : miskin
gratis : gratis
mengelilingi : keliling
menyantap : santap
khas : khas
nasi : nasi
jamblang : jamblang
pelabuhan : labuh
gagas : gagas
bazn : bazn
jabar : jabar
minions : minions
nunung : nunung
rafathar : rafathar
live : live


...

brando : brando
kelanjutan : lanjut
bani : bani
prustasi : prustasi
sllu : sllu
diselengga : diselengga
mengampuni : ampun
meraka : raka
meray : meray
sempatnya : sempat
kepikiran : pikir
tugasnya : tugas
mendunia : dunia
henry : henry
cavill : cavill
pari : pari
intan : intan
permata : permata
sari : sari
lapak : lapak
hetero : hetero
space : space
kirain : kirain
boongan : boongan
dijadikan : jadi
doain : doain
penyidik : sidik
parah : parah
kredibel : kredibel
pemberitaan : berita
busuk : busuk
kelebihan : lebih
menaikkan : naik
super : super
canggih : canggih
berjudul : judul
dipersiapkan : siap
diseluruh : seluruh
pemimpim : pemimpim
ciptakan : cipta
bangkit : bangkit
moen : moen
alfatihah : alfatihah
teladan : teladan
menawarkan : tawar
visio : visio
beban : beban
coblos : coblos
lodewijk : lodewijk
paulus : paulus
maimun : maimun
paktelur : paktelur
adia : adia
haus : haus
jabatan : jabat
berbisik : bisik
jokow : jokow
kalahkan : kalah
agusyu : agusyu
agusyudhoyono : agusyudhoyo

medi : medi
mener : ter
revitalisasi : revitalisasi
bodo : bodo
tetep : tetep
hoby : hoby
menutupi : tutup
kegagalan : gagal
bangganya : bangga
sopan : sopan
menuduh : tuduh
berpolitik : politik
menga : menga
plastik : plastik
meneba : meneba
giat : giat
eksistensi : eksistensi
citra : citra
berkaitan : kait
menjad : menjad
nyapres : nyapres
kebanggaan : bangga
dibesarkan : besar
ayahnya : ayah
denger : denger
maimoen : maimoen
mojokerto : mojokerto
nyicipin : nyicipin
kurma : kurma
kacang : kacang
secangkir : cangkir
zamzam : zamzam
penga : penga
pelaksana : laksana
muhamad : muhamad
maunya : mau
muak : muak
buntelan : buntel
kentut : kentut
pengecekan : kece
diperbaiki : baik
laporan : lapor
woro : woro
widowati : widowati
siapkan : siap
kerusakan : rusa
pracimantoro : pracimantoro
smpn : smpn
giriwoyo : giriwoyo
ruma : ruma
metereologi : metereologi
klimatologi : klimatologi
geofisika : geofisika
bmkg : bmkg
korita : korita
jajaran : jajar
distribusi : distribusi
pengumuman : umum
b

basuwedan : basuwedan
salahuddin : salahuddin
konduktivitas : konduktivitas
tarik : tarik
pemberani : berani
memecat : pecat
lalunya : lalu
ditiru : tiru
menyodorkan : sodor
penutupnya : tutup
bergambar : gambar
perbincangan : bincang
padat : padat
hubungannya : hubung
sedangka : dangka
mantaap : mantaap
bangeet : bangeet
perka : perka
milineal : milineal
mempertanggung : tanggung
jawabkan : jawab
buatnya : buat
keberlanjutan : lanjut
sentris : sentris
aryo : aryo
seno : seno
bagaskoro : bagaskoro
kaku : kaku
bertele : tele
pilu : pilu
konvoi : konvoi
militer : militer
ndan : ndan
enih : enih
arogan : arogan
ancam : ancam
yudo : yudo
margono : margono
bela : bela
buahnya : buah
tower : tower
diwarnai : warna
memasang : pasang
funakawan : funakawan
kusnanto : kusnanto
anggoro : anggoro
hoaks : hoaks
memanipulasi : manipulasi
senantiasa : senantiasa
suda : suda
kapuspen : kapuspen
pencopotan : copot
banner : banner
prosedur : prosedur
netralitas : netralitas
selengkapnya : lengkap
pembek

kehen : kehen
tambahan : tambah
hitamputihganjar : hitamputihganjar
jala : jala
nyaman : nyaman
rapi : rapi
terapkan : terap
pola : pola
pujian : puji
perihal : perihal
ubanan : uban
ciptaan : cipta
yenny : yenny
wahid : wahid
orgnya : orgnya
tentuny : tentuny
dipilihnya : pilih
sebagau : bagau
mening : ning
abis : abis
deng : deng
tertinggal : tinggal
sulung : sulung
mengerucut : kerucut
purwodadi : purwodadi
bulatkan : bulat
ngasih : ngasih
dadakan : dada
pemanfaatan : manfaat
imin : imin
segar : segar
merap : rap
gelem : gelem
susi : sus
nasdemajur : nasdemajur
aniesgaklaku : aniesgaklaku
gantungmatihakimmk : gantungmatihakimmk
gantungma : gantungma
jurkamnya : jurkamnya
bapaknya : bapak
mosok : mosok
ngobrolnya : ngobrolnya
once : once
mekel : mekel
mimin : mimin
didaftar : daftar
produktif : produktif
pilpr : pilpr
perbatasan : batas
legokjawa : legokjawa
panga : panga
dikaji : kaji
lurik : lurik
kalinya : kali
mengerut : kerut
masukan : masuk
koreksi : koreksi
kendeng : kendeng
t

andy : andy
noya : noya
religius : religius
mengayomi : ayom
jenderan : jenderan
kalsel : kalsel
lebarkan : lebar
sayap : sayap
pemasaran : pasar
rekor : rekor
pemeri : meri
menyebalkan : sebal
dibikin : bikin
geleng : geleng
neko : neko
menya : menya
digalang : galang
penentuan : tentu
proactive : proactive
pendengar : dengar
bida : bida
kocak : kocak
bonge : bonge
citayam : citayam
fashion : fashion
week : week
berjarak : jarak
maruarar : maruarar
jhon : jhon
bongkar : bongkar
diselesaikan : selesai
gembleng : gembleng
mengat : mengat
edan : edan
menyam : menyam
nikah : nikah
mengurangi : kurang
mengabdi : abdi
{'memperkuat': 'kuat', 'pondasi': 'pondasi', 'kemenangan': 'menang', 'sahabat': 'sahabat', 'depok': 'depok', 'berkunjung': 'kunjung', 'kota': 'kota', 'semoga': 'moga', 'melanjutkan': 'lanjut', 'kepemimpinan': 'pimpin', 'nasional': 'nasional', 'presiden': 'presiden', 'joko': 'joko', 'widodo': 'widodo', 'belajar': 'ajar', 'reformasi': 'reformasi', 'birokrasi': 'birokrasi', 'mena

In [11]:
df['stemming'] = stem_done
print (df['stemming'])

0       [kuat, pondasi, menang, sahabat, depok, kunjun...
1       [moga, lanjut, pimpin, nasional, presiden, jok...
2       [ajar, reformasi, birokrasi, menangtotal, becc...
3       [kuat, pondasi, menang, sahabat, depok, kunjun...
4       [ajar, reformasi, birokrasi, menangtotal, becc...
                              ...                        
3908    [urus, edan, urus, pasu, sopir, becak, kota, b...
3909    [rawan, gapura, nusantara, purnawirawan, polri...
3910    [dukung, galang, sahabatganjar, menang, giat, ...
3911    [gubernur, jateng, remaja, nikah, muda, salah,...
3912     [ayah, presiden, semangat, abdi, bangun, negeri]
Name: stemming, Length: 3913, dtype: object


# UNTOKENIZE

In [12]:
def untokenize(words):
    """
    Untokenizing a text undoes the tokenizing operation, restoring
    punctuation and spaces to the places that people expect them to be.
    Ideally, `untokenize(tokenize(text))` should be identical to `text`,
    except for line breaks.
    """
    text = ' '.join(words)
    step1 = text.replace("`` ", '"').replace(" ''", '"').replace('. . .',  '...')
    step2 = step1.replace(" ( ", " (").replace(" ) ", ") ")
    step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
    step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
    step5 = step4.replace(" '", "'").replace(" n't", "n't").replace(
         "can not", "cannot")
    step6 = step5.replace(" ` ", " '")
    return step6.strip()

In [13]:
df['detokens']=df['stemming'].apply(lambda x: untokenize(x))
df

,id,date,place,screen_name,text,cleansing,case fold,stopwords,stemming,detokens
0,1661196268318920705,2023-05-24 02:23:48,Indonesia,IDSahabatGanjar,Untuk memperkuat pondasi kemenangan untuk Ganj...,Untuk memperkuat pondasi kemenangan untuk Ganj...,untuk memperkuat pondasi kemenangan untuk ganj...,"[memperkuat, pondasi, kemenangan, sahabat, dep...","[kuat, pondasi, menang, sahabat, depok, kunjun...",kuat pondasi menang sahabat depok kunjung kota...
1,1661196173997395969,2023-05-24 02:23:26,"DKI Jakarta, Indonesia",DiniHdianti,"RT @LaingenLisa: ""Semoga Ganjar Pranowo bisa m...",Semoga Ganjar Pranowo bisa melanjutkan kepemim...,semoga ganjar pranowo bisa melanjutkan kepemim...,"[semoga, melanjutkan, kepemimpinan, nasional, ...","[moga, lanjut, pimpin, nasional, presiden, jok...",moga lanjut pimpin nasional presiden joko widodo
2,1661196167450099714,2023-05-24 02:23:24,Jakarta,ShintaSusan2,RT @ShintaSusan2: Belajar Reformasi Birokrasi ...,Belajar Reformasi Birokrasi bersama Ganjar Pra...,belajar reformasi birokrasi bersama ganjar pra...,"[belajar, reformasi, birokrasi, menangtotal, b...","[ajar, reformasi, birokrasi, menangtotal, becc...",ajar reformasi birokrasi menangtotal becca has...
3,1661196146767962116,2023-05-24 02:23:19,NaN,sahabatganjar,Untuk memperkuat pondasi kemenangan untuk Ganj...,Untuk memperkuat pondasi kemenangan untuk Ganj...,untuk memperkuat pondasi kemenangan untuk ganj...,"[memperkuat, pondasi, kemenangan, sahabat, dep...","[kuat, pondasi, menang, sahabat, depok, kunjun...",kuat pondasi menang sahabat depok kunjung kota...
4,1661196142363938817,2023-05-24 02:23:18,Jakarta,ShintaSusan2,Belajar Reformasi Birokrasi bersama Ganjar Pra...,Belajar Reformasi Birokrasi bersama Ganjar Pra...,belajar reformasi birokrasi bersama ganjar pra...,"[belajar, reformasi, birokrasi, menangtotal, b...","[ajar, reformasi, birokrasi, menangtotal, becc...",ajar reformasi birokrasi menangtotal becca has...
...,...,...,...,...,...,...,...,...,...,...
3908,1686005227399651328,2023-07-31 13:25:45,Solo ~ Jawa tengah ~ Indonesia,sussssie,RT @lek_brewok: Urusan ama wong edan nanti aja...,Urusan wong edan nanti masih urusan sama Pasuk...,urusan wong edan nanti masih urusan sama pasuk...,"[urusan, edan, urusan, pasukan, sopir, becak, ...","[urus, edan, urus, pasu, sopir, becak, kota, b...",urus edan urus pasu sopir becak kota bogor dukung
3909,1686005150635425792,2023-07-31 13:25:27,Indonesia,wil_ws_,RT @Jumianto_RK: Relawan Gapura Nusantara (RGN...,Relawan Gapura Nusantara Purnawirawan Polri me...,relawan gapura nusantara purnawirawan polri me...,"[relawan, gapura, nusantara, purnawirawan, pol...","[rawan, gapura, nusantara, purnawirawan, polri...",rawan gapura nusantara purnawirawan polri duku...
3910,1686005038316134400,2023-07-31 13:25:00,NaN,Jelli_cent,Dukungan jg perlu digalang.\nItu yg terus dila...,Dukungan perlu digalang terus dilakukan Sahaba...,dukungan perlu digalang terus dilakukan sahaba...,"[dukungan, digalang, sahabatganjar, memenangka...","[dukung, galang, sahabatganjar, menang, giat, ...",dukung galang sahabatganjar menang giat positif
3911,1686005014840709120,2023-07-31 13:24:54,"Semarang Barat, Indonesia",wongkalibanteng,"RT @03__nakula: Gubernur Jateng, Ganjar Pranow...",Gubernur Jateng Ganjar Pranowo minta agar para...,gubernur jateng ganjar pranowo minta agar para...,"[gubernur, jateng, remaja, nikah, muda, salah,...","[gubernur, jateng, remaja, nikah, muda, salah,...",gubernur jateng remaja nikah muda salah upaya ...


In [14]:
df.to_csv('ganjar_prepro.csv')